<a href="https://colab.research.google.com/github/Sela80/s/blob/main/Donn%C3%A9es_sur_les_ventes_au_d%C3%A9tail_mondiales_commandes%2C_avis_et_tendances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Préparation des Données

**a. Importation des Bibliothèques**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

**b. Chargement des Données**

In [ ]:
data=pd.read_csv('/content/merch_sales.csv')

In [ ]:
data.head()

In [ ]:
data.describe().transpose()

**c. Nettoyage des Données**

1. Vérifier les valeurs manquantes et les types de données.
2. Convertir les colonnes de dates au format datetime.
3. Traiter les doublons et les valeurs aberrantes.

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:

# Convertir les colonnes de dates au format datetime
date_columns = ['Order Date']  # Remplacez par les noms de vos colonnes de date
for col in date_columns:
    data[col] = pd.to_datetime(data[col])


In [ ]:
data.duplicated().sum()

In [ ]:
# Visualisation des valeurs aberrantes avec des boxplots
numerical_cols = data.select_dtypes(include=['number']).columns  # Sélectionner les colonnes numériques
for col in numerical_cols:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot de {col}')
    plt.show()


A travers nos Boxplots il nous semble que ce sont nos colonnes "Sales per Unit", "Quantity", "Total Sales" qui contiennent des valeurs aberrantes.

In [ ]:
# Appliquer une transformation logarithmique aux colonnes avec valeurs aberrantes
for col in ['Sales per Unit', 'Quantity', 'Total Sales']:
    # Ajouter 1 pour éviter le logarithme de zéro
    data[col + '_log'] = np.log1p(data[col])

# Visualiser les boxplots après transformation logarithmique
for col in ['Sales per Unit_log', 'Quantity_log', 'Total Sales_log']:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=data[col])
    plt.title(f'Boxplot de {col} (après transformation logarithmique)')
    plt.show()

# 2. Analyse Exploratoire des Données (EDA)

**a. Visualisation des Ventes**


*   Créer des graphiques pour visualiser les ventes totales au fil du temps.



In [ ]:
sales_over_time = data.groupby('Order Date')['Total Sales'].sum()
plt.figure(figsize=(12, 6))
sales_over_time.plot()
plt.title('Ventes Totales au Fil du Temps')
plt.xlabel('Date')
plt.ylabel('Ventes Totales')
plt.show()

# b. Analyse des Avis

**Analyser les sentiments des avis.**

In [ ]:
import plotly.express as px
fig = px.histogram(data, x="Review", title="Distribution of Reviews")
fig.show()

# 3. Analyse des Tendances

**a. Ventes par Catégorie de Produit**

In [ ]:
category_sales = data.groupby('Product Category')['Total Sales'].sum().sort_values()
fig = px.bar(category_sales, y=category_sales.index, x='Total Sales', title='Ventes par Catégorie de Produit')
fig.show()

**b. Ventes par Genre**

In [ ]:
gender_sales = data.groupby('Buyer Gender')['Total Sales'].sum()
fig = px.pie(data, values='Total Sales', names='Buyer Gender', title='Ventes par Genre')
fig.show()

# 4. Modélisation Prédictive

**a. Préparation des Données pour la Modélisation**


* Créer des fonctionnalités basées sur le temps (jour, mois, année).
* Créer des variables d'entrée et de sortie pour l'apprentissage automatique.




In [ ]:
data['Year'] = data['Order Date'].dt.year
data['Month'] = data['Order Date'].dt.month
X = data[['Year', 'Month']]
y = data['Total Sales']

**b. Division des Données**
* Diviser les données en ensembles de formation et de test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**c. Choix du Modèle**
*  Utiliser un modèle de régression (par exemple, RandomForestRegressor).

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()
model.fit(X_train, y_train)

**d. Prédiction et Évaluation**

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

predictions = model.predict(X_test)
print(f'MSE: {mean_squared_error(y_test, predictions)}')
print(f'MAE: {mean_absolute_error(y_test, predictions)}')

## 5. Visualisation des Prédictions
* Comparer les prédictions aux valeurs réelles.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(y_test.values, label='Ventes Réelles')
plt.plot(predictions, label='Prédictions', alpha=0.7)
plt.title('Comparaison des Ventes Réelles et Prédites')
plt.xlabel('Index')
plt.ylabel('Ventes Totales')
plt.legend()
plt.show()

In [ ]:
results_df = pd.DataFrame({'Actual Sales': y_test.values, 'Predicted Sales': predictions})

# Display the DataFrame
results_df